# Visualización de trayectoria de esfuerzos y superficie de fluencia de von Mises en el espacio de esfuerzos principales o de Haigh-Westergaard


```
Por: Michael Heredia Pérez
Ayudado por: Gemini 2.5 Pro
email: mherediap@unal.edu.co
fecha: junio 2 de 2025
```

Este notebook demuestra cómo graficar una trayectoria de esfuerzos principales en el espacio de Haigh-Westergaard y superponer la superficie de fluencia de von Mises. Esto nos permite visualizar si un estado de esfuerzos alcanza el límite elástico de un material dúctil.

**Conceptos Clave:**
1.  **Espacio de Esfuerzos Principales (Haigh-Westergaard):** Un espacio 3D donde los ejes son los tres esfuerzos principales ($\sigma_1, \sigma_2, \sigma_3$).
2.  **Trayectoria de Carga:** La curva trazada en el espacio de Haigh-Westergaard por los esfuerzos principales de un punto a medida que varían las cargas externas aplicadas al cuerpo.
3.  **Superficie de Fluencia de von Mises:** Para un material isótropo, define el límite entre el comportamiento elástico y el plástico. Es un cilindro circular infinito cuyo eje es la línea hidrostática ($\sigma_1 = \sigma_2 = \sigma_3$). Su ecuación es:
    $(\sigma_1 - \sigma_2)^2 + (\sigma_2 - \sigma_3)^2 + (\sigma_3 - \sigma_1)^2 = 2 \sigma_y^2$
    donde $\sigma_y$ es el esfuerzo de fluencia uniaxial.
4.  **Fluencia:** Ocurre cuando la trayectoria de carga toca la superficie de fluencia.

Utilizaremos `numpy` para cálculos y `plotly` para gráficos 3D interactivos.

In [2]:
import numpy as np
import plotly.graph_objects as go

## Parte 1: Definición de Funciones y Parámetros

Primero, definimos una función para calcular los esfuerzos principales a partir de los componentes del tensor de esfuerzos. Luego, definimos cómo variarán los componentes del tensor de esfuerzos en función del tiempo `t`. Finalmente, establecemos los parámetros para la simulación temporal y las propiedades del material (esfuerzo de fluencia $\sigma_y$).

In [4]:
# --- Función para calcular esfuerzos principales ---
def get_principal_stresses(sigma_xx, sigma_yy, sigma_zz, tau_xy, tau_yz, tau_xz):
    """
    Calcula los esfuerzos principales para un estado de esfuerzos 3D.
    Retorna un array con los tres valores propios.
    """
    stress_tensor = np.array([
        [sigma_xx, tau_xy,   tau_xz],
        [tau_xy,   sigma_yy, tau_yz],
        [tau_xz,   tau_yz,   sigma_zz]
    ])
    principal_stresses = np.linalg.eigvalsh(stress_tensor) # Para tensores simétricos
    return principal_stresses

# --- Definición de los Esfuerzos como Funciones del Tiempo (t) ---
# ¡Experimenta cambiando estas funciones!
def sigma_xx_func(t):
    return 50 * np.sin(t/5) + 20 * t

def sigma_yy_func(t):
    return 30 * np.cos(t/3) + 5*t # Añadida una componente lineal

def sigma_zz_func(t):
    return 10 * np.sin(t/2)

def tau_xy_func(t):
    return 40 * np.sin(t/4) * np.cos(t/8) - 8*t # Añadida una componente lineal

def tau_yz_func(t):
    return 15 * np.cos(t/6)

def tau_xz_func(t):
    return 25 * np.sin(t/7)

# --- Parámetros de Simulación y Material ---
t_start = 0
t_end = 2 * np.pi * 1.5 # Duración de la simulación
n_steps_trajectory = 200 # Puntos en la trayectoria
t_values = np.linspace(t_start, t_end, n_steps_trajectory)

sigma_y = 150 # Esfuerzo de fluencia del material (e.g., MPa)
                # Asegúrate de que las unidades sean consistentes

## Parte 2: Cálculo de la Trayectoria de Esfuerzos Principales

Ahora, iteramos a través del tiempo, calculamos los componentes del tensor de esfuerzos en cada instante `t`, y luego obtenemos los tres esfuerzos principales. Para facilitar la visualización con la superficie de fluencia, ordenaremos los esfuerzos principales ($\sigma_1 \ge \sigma_2 \ge \sigma_3$) antes de almacenarlos, aunque esto puede introducir las discontinuidades visuales discutidas si el orden intrínseco de los valores propios cambia.

In [5]:
# Listas para almacenar los caminos de los tres esfuerzos principales (ordenados)
path_s1 = [] # Mayor esfuerzo principal
path_s2 = [] # Esfuerzo principal intermedio
path_s3 = [] # Menor esfuerzo principal

for t in t_values:
    # Calcular los componentes del tensor de esfuerzos en el tiempo t
    s_xx = sigma_xx_func(t)
    s_yy = sigma_yy_func(t)
    s_zz = sigma_zz_func(t)
    t_xy = tau_xy_func(t)
    t_yz = tau_yz_func(t)
    t_xz = tau_xz_func(t)

    # Obtener los tres esfuerzos principales (valores propios)
    principal_stresses_at_t = get_principal_stresses(s_xx, s_yy, s_zz, t_xy, t_yz, t_xz)

    # Ordenar los esfuerzos principales: sigma1 >= sigma2 >= sigma3
    ordered_stresses = np.sort(principal_stresses_at_t)[::-1] # Orden descendente

    path_s1.append(ordered_stresses[0])
    path_s2.append(ordered_stresses[1])
    path_s3.append(ordered_stresses[2])

# Convertir listas a arrays de numpy
path_s1 = np.array(path_s1)
path_s2 = np.array(path_s2)
path_s3 = np.array(path_s3)

print(f"Primeros 5 puntos de la trayectoria (s1, s2, s3):")
for i in range(min(5, len(path_s1))):
    print(f"t={t_values[i]:.2f}: ({path_s1[i]:.2f}, {path_s2[i]:.2f}, {path_s3[i]:.2f})")

Primeros 5 puntos de la trayectoria (s1, s2, s3):
t=0.00: (36.21, 0.00, -6.21)
t=0.05: (36.45, 1.42, -5.98)
t=0.09: (36.68, 2.85, -5.75)
t=0.14: (36.90, 4.27, -5.52)
t=0.19: (37.12, 5.69, -5.29)


## Parte 3: Generación de Puntos para la Superficie de Fluencia de von Mises

La superficie de von Mises es un cilindro. Para graficarlo, lo parametrizamos usando coordenadas cilíndricas $(\xi, \rho, \theta_{cyl})$ en el sistema de Haigh-Westergaard, donde $\xi$ es la coordenada a lo largo del eje hidrostático, $\rho = \sqrt{2/3} \sigma_y$ es el radio constante del cilindro, y $\theta_{cyl}$ es el ángulo alrededor del eje. Luego, transformamos estos puntos de vuelta al espacio $(\sigma_1, \sigma_2, \sigma_3)$ usando las ecuaciones de transformación (similares a la Ecuación 16.21 del libro de texto).

In [6]:
# Radio del cilindro de von Mises en el plano desviador (plano pi)
rho_vm = np.sqrt(2/3) * sigma_y

# Rango para la coordenada hidrostática (xi de Haigh-Westergaard)
# Se ajusta para cubrir la extensión de la trayectoria y un poco más
min_coord_sum = np.min(path_s1 + path_s2 + path_s3)
max_coord_sum = np.max(path_s1 + path_s2 + path_s3)

# xi_HW = (s1+s2+s3)/sqrt(3)
xi_min = min_coord_sum / np.sqrt(3) - 1.5 * rho_vm # Extender un poco
xi_max = max_coord_sum / np.sqrt(3) + 1.5 * rho_vm
n_xi_points = 30  # Número de puntos a lo largo del eje del cilindro
xi_vals_hw = np.linspace(xi_min, xi_max, n_xi_points)

# Ángulo alrededor del eje del cilindro (0 a 2*pi para un cilindro completo)
n_theta_points = 60 # Número de puntos alrededor de la circunferencia
theta_cyl_vals = np.linspace(0, 2 * np.pi, n_theta_points)

# Crear la malla para el cilindro
XI_HW, THETA_CYL = np.meshgrid(xi_vals_hw, theta_cyl_vals)

# Transformación de coordenadas cilíndricas de Haigh-Westergaard (xi, rho, theta) a (s1, s2, s3)
# s_hidro_component = XI_HW / np.sqrt(3) # Esto es (s1+s2+s3)/3
# s1_surf = s_hidro_component + rho_vm * np.sqrt(2/3) * np.cos(THETA_CYL)
# s2_surf = s_hidro_component + rho_vm * np.sqrt(2/3) * np.cos(THETA_CYL - 2 * np.pi / 3)
# s3_surf = s_hidro_component + rho_vm * np.sqrt(2/3) * np.cos(THETA_CYL + 2 * np.pi / 3)

# Usando la ecuación 16.21 del libro (adaptada):
# sigma_i = xi_HW/sqrt(3) + sqrt(2/3)*rho*cos(theta_lode - (i-1)*120_grados)
# Aquí, nuestro rho es rho_vm y theta_lode es THETA_CYL.
s1_surf = (XI_HW / np.sqrt(3)) + rho_vm * np.sqrt(2/3) * np.cos(THETA_CYL)
s2_surf = (XI_HW / np.sqrt(3)) + rho_vm * np.sqrt(2/3) * np.cos(THETA_CYL - 2 * np.pi / 3)
s3_surf = (XI_HW / np.sqrt(3)) + rho_vm * np.sqrt(2/3) * np.cos(THETA_CYL + 2 * np.pi / 3)

print(f"Radio del cilindro de von Mises (rho_vm): {rho_vm:.2f}")
print(f"Rango de xi para el cilindro: [{xi_min:.2f}, {xi_max:.2f}]")

Radio del cilindro de von Mises (rho_vm): 122.47
Rango de xi para el cilindro: [-166.39, 324.11]


## Parte 4: Graficación 3D Interactiva

Finalmente, usamos `plotly.graph_objects` para crear el gráfico 3D. Mostraremos la trayectoria de carga y la superficie de fluencia de von Mises. La interactividad de Plotly permitirá rotar, hacer zoom y explorar el gráfico.

In [7]:
fig = go.Figure()

# Añadir la superficie de fluencia de von Mises
fig.add_trace(go.Surface(
    x=s1_surf, y=s2_surf, z=s3_surf,
    opacity=0.4, # Hacerla semitransparente
    colorscale='Greys',
    showscale=False,
    name=f'Superficie von Mises ($\sigma_y$={sigma_y})',
    hoverinfo='skip' # No mostrar tooltips para la superficie (opcional)
))

# Añadir la trayectoria de carga
fig.add_trace(go.Scatter3d(
    x=path_s1, y=path_s2, z=path_s3,
    mode='lines+markers',
    name='Trayectoria de Carga',
    line=dict(color='royalblue', width=4),
    marker=dict(size=3, color=t_values, colorscale='Viridis', opacity=0.7,
                colorbar=dict(title='Tiempo (t)', thickness=10, x=0)), # Añadir colorbar para el tiempo
    customdata=t_values, # Añadir valores de t para el hover
    hovertemplate='<b>t: %{customdata:.2f}</b><br>σ1: %{x:.2f}<br>σ2: %{y:.2f}<br>σ3: %{z:.2f}<extra></extra>'
))

# Marcar el inicio y el fin de la trayectoria
fig.add_trace(go.Scatter3d(
    x=[path_s1[0]], y=[path_s2[0]], z=[path_s3[0]],
    mode='markers', name='Inicio (t=0)',
    marker=dict(size=8, color='limegreen', symbol='circle'),
    hovertemplate='<b>Inicio</b><br>σ1: %{x:.2f}<br>σ2: %{y:.2f}<br>σ3: %{z:.2f}<extra></extra>'
))
fig.add_trace(go.Scatter3d(
    x=[path_s1[-1]], y=[path_s2[-1]], z=[path_s3[-1]],
    mode='markers', name=f'Fin (t={t_end:.2f})',
    marker=dict(size=8, color='crimson', symbol='x'),
    hovertemplate='<b>Fin</b><br>σ1: %{x:.2f}<br>σ2: %{y:.2f}<br>σ3: %{z:.2f}<extra></extra>'
))

# Añadir la línea hidrostática (opcional, para referencia)
line_hidro_limit = max(abs(xi_min), abs(xi_max)) * np.sqrt(3) * 0.6 # Escalar para visualización
fig.add_trace(go.Scatter3d(
    x=[-line_hidro_limit, line_hidro_limit],
    y=[-line_hidro_limit, line_hidro_limit],
    z=[-line_hidro_limit, line_hidro_limit],
    mode='lines', name='Línea Hidrostática (σ1=σ2=σ3)',
    line=dict(color='black', width=2, dash='dash')
))


# Configuración de la apariencia del gráfico y los ejes
fig.update_layout(
    title_text='Trayectoria de Carga y Superficie de Fluencia de von Mises',
    title_x=0.5,
    scene=dict(
        xaxis_title_text='$\sigma_1$',
        yaxis_title_text='$\sigma_2$',
        zaxis_title_text='$\sigma_3$',
        aspectmode='data', # Mantiene las proporciones correctas
        # Los rangos se ajustan automáticamente por defecto, pero se pueden fijar si es necesario
        # xaxis_range=[min(s1_surf.min(), path_s1.min()), max(s1_surf.max(), path_s1.max())],
        # yaxis_range=[min(s2_surf.min(), path_s2.min()), max(s2_surf.max(), path_s2.max())],
        # zaxis_range=[min(s3_surf.min(), path_s3.min()), max(s3_surf.max(), path_s3.max())],
        camera_eye=dict(x=1.8, y=1.8, z=0.8) # Ajustar la vista inicial de la cámara
    ),
    legend_title_text='Leyenda',
    margin=dict(l=10, r=10, b=10, t=50), # Ajustar márgenes
    height=700 # Altura del gráfico
)

fig.show()

Fin :)